# Learn Posture

use machine learning to recognize robot's posture (following the example in [scikit-learn-intro.ipynb](./scikit-learn-intro.ipynb) )

## 1. Data collection

We have colleceted data before, you need to add new data if you want to add new posture.

* the dateset are in *robot_pose_data* folder
* each file contains the data belongs to this posture, e.g. the data in *Back* file are collected when robot was in "Back" posture
* the data file can be load by ```pickle```, e.g. ```pickle.load(open('Back'))```, the data is a list of feature data
* the features (e.g. each row of the data) are ['LHipYawPitch', 'LHipRoll', 'LHipPitch', 'LKneePitch', 'RHipYawPitch', 'RHipRoll', 'RHipPitch', 'RKneePitch', 'AngleX', 'AngleY'], where 'AngleX' and 'AngleY' are body angle (e.g. ```Perception.imu```) and others are joint angles.

## 2. Data preprocessing

In [2]:
%pylab inline
import pickle
from os import listdir, path
import numpy as np
from sklearn import svm, metrics

ROBOT_POSE_DATA_DIR = 'robot_pose_data'

Populating the interactive namespace from numpy and matplotlib


In [3]:
classes = listdir(ROBOT_POSE_DATA_DIR)
print classes

['Frog', 'Right', 'StandInit', 'Left', 'Belly', 'Stand', 'Sit', 'Crouch', 'HeadBack', 'Back', 'Knee']


In [4]:
def load_pose_data(i):
    '''load pose data from file'''
    data = []
    target = []
    # YOUR CODE HERE
    
    filename = path.join(ROBOT_POSE_DATA_DIR, classes[i])
    data = pickle.load(open(filename))
    target = [i] * len(data)
    return data, target

In [5]:
# load all the data
all_data = []
all_target = []
# YOUR CODE HERE

for i in range(len(classes)):
    temp_data, temp_target = load_pose_data(i)
    for j in range(len(temp_data)):
        all_data.append(temp_data[j])
    for j in range(len(temp_target)):
        all_target.append(temp_target[j])
        
all_data = np.array(all_data)
all_target = np.array(all_target)
"""
MY

all_data = np.array([[]])
all_target = np.array([])
# YOUR CODE HERE

for i in range(len(classes)):
    temp_data, temp_target = load_pose_data(i)
    for j in range(len(temp_data)):
        all_data = np.append(all_data, temp_data[j])
    for j in range(len(temp_target)):
        all_target = np.append(all_target, temp_target[j])

FRED


all_data2 = []
all_target2 = []
for i in range(len(classes)):
    temp_data, temp_target = load_pose_data(i)
    for val in enumerate(temp_data):
        all_data2.append(val[1])
    for val in enumerate(temp_target):
        all_target2.append(val[1])

all_data_array = np.ndarray((len(all_data2[0]),len(all_data2)))
all_target_array = np.ndarray(len(all_target2))

for i in range(len(all_data)):
    all_data_array[:,i] = all_data2[i]
for i in range(len(all_target)):
    all_target_array[i] = all_target2[i]
    
all_data_array = all_data_array.T

all_data2 = all_data_array
all_target2 = all_target_array

print 'LOOOOOOOOOOOOOOOOOOOOL'
print all_data
print all_data2
print all_data == all_data2
"""
print all_data

print 'total number of data', len(all_data)

[[-0.40953612 -0.19477606 -1.57997811 ...,  2.13997221 -0.02792501
   0.98960161]
 [-0.40953612 -0.19631004 -1.58151209 ...,  2.13997221 -0.03490639
   0.99134684]
 [-0.40953612 -0.19477606 -1.58151209 ...,  2.13997221 -0.03490639
   0.99483728]
 ..., 
 [ 0.30377388  0.01538205  0.42035794 ...,  1.71198606  0.         -0.08726692]
 [ 0.30377388  0.01538205  0.42035794 ...,  1.71198606  0.         -0.08726692]
 [ 0.30377388  0.01538205  0.42035794 ...,  1.71198606  0.         -0.08726692]]
total number of data 222


In [6]:
# shuffule data
permutation = np.random.permutation(len(all_data))
n_training_data = int(len(all_data) * 0.7)
training_data = permutation[:n_training_data]

## 3. Learn on training data

In scikit-learn, an estimator for classification is a Python object that implements the methods fit(X, y) and predict(T). An example of an estimator is the class sklearn.svm.SVC that implements support vector classification.

In [7]:
clf = svm.SVC(gamma=0.001, C=100.)

### learning

In [8]:
clf.fit(all_data, all_target) 

SVC(C=100.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape=None, degree=3, gamma=0.001, kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

### predicting

In [9]:
exp, res = clf.predict(all_data), all_target

In [10]:
def evaluate(expected, predicted):
    print("Classification report:\n%s\n" % metrics.classification_report(expected, predicted))

    print("Confusion matrix:\n%s" % metrics.confusion_matrix(expected, predicted))

In [12]:
expected = exp
predicted = res
# YOUR CODE HERE

evaluate(expected, predicted)

Classification report:
             precision    recall  f1-score   support

          0       1.00      1.00      1.00        10
          1       0.91      1.00      0.95        10
          2       1.00      1.00      1.00        52
          3       1.00      1.00      1.00        20
          4       1.00      1.00      1.00        19
          5       1.00      1.00      1.00        11
          6       1.00      1.00      1.00        26
          7       1.00      0.97      0.98        31
          8       1.00      1.00      1.00        10
          9       1.00      1.00      1.00        23
         10       1.00      1.00      1.00        10

avg / total       1.00      1.00      1.00       222


Confusion matrix:
[[10  0  0  0  0  0  0  0  0  0  0]
 [ 0 10  0  0  0  0  0  0  0  0  0]
 [ 0  0 52  0  0  0  0  0  0  0  0]
 [ 0  0  0 20  0  0  0  0  0  0  0]
 [ 0  0  0  0 19  0  0  0  0  0  0]
 [ 0  0  0  0  0 11  0  0  0  0  0]
 [ 0  0  0  0  0  0 26  0  0  0  0]
 [ 0  1  0  0 

## 4. Evaluate on the test data

In [13]:
expected = []
predicted = []
# YOUR CODE HERE

evaluate(expected, predicted)

ValueError: max() arg is an empty sequence

## 5. Deploy to the real system

We can simple use `pickle` module to serialize the trained classifier.

In [1]:
import pickle
ROBOT_POSE_CLF = 'robot_pose.pkl'
pickle.dump(clf, open(ROBOT_POSE_CLF, 'w'))

NameError: name 'clf' is not defined

Then, in the application we can load the trained classifier again.

In [71]:
clf2 = pickle.load(open(ROBOT_POSE_CLF))
clf2.predict(all_data[-1]), all_target[-1]

(array([10]), 10)